**Script to use Reddit API and scrape data**

In [28]:
# imports
import pandas as pd
import requests
import time
import getpass

from bs4 import BeautifulSoup

---
**Access Info**

In [30]:
client_id = getpass.getpass()       #alphanumeric string provided under "personal use script"
client_secret = getpass.getpass()   #alphanumeric string provided as "secret"
user_agent = getpass.getpass()      #the name of your application
username = getpass.getpass()        #your reddit username
password =  getpass.getpass()       #your reddit password

 ········
 ········
 ········
 ········
 ········


Now we're on our way to retrieving our access token; we'll use the basic authentication framework to get there.

In [34]:
auth = requests.auth.HTTPBasicAuth(client_id, client_secret)

data = {
    'grant_type': 'password',
    'username': username,
    'password': password
}

#-----------------------------------------------------------------
headers = {'User-Agent': 'dsb0826project3/0.0.1'}

response = requests.post(
    'https://www.reddit.com/api/v1/access_token',
    auth = auth,
    data = data,
    headers = headers)

#--- confirm response ----------------------------------------------
print(response)

<Response [200]>


Hopefully upon running the above, you received a successful response code and can save your token. These should last for about two hours by default.

In [36]:
#retrieve access token
token = response.json()['access_token']

headers['Authorization'] = f'bearer {token}'

requests.get('https://oauth.reddit.com/api/v1/me', headers = headers).status_code == 200

True

Now let's add your access token to the headers and verify that you can successfully submit a call to the api.

If all went correctly, we can finally create a simple request.

In [40]:
base_url = 'https://oauth.reddit.com/r/'
subreddit = 'travel'

params = {
    'limit': 100,
    'after': last
}

response = requests.get(base_url+subreddit,
                        headers = headers,
                        params = params)

Explore the response object. Where is our submission data? How many posts were retrieved by default?

In [46]:
response.status_code

200

**Parsing Function**

In [272]:
def parse_data(response, output_path = '../data/reddit_realestate.csv'):
    
    batch = response.json()['data']['children']

    data = []

    for item in batch:
        post = {
            'post_id' : item['data']['name'],
            'post_title' : item['data']['title'],
            'post_text' : item['data']['selftext']
        }
        data.append(post)
    
    
    df = pd.DataFrame(data)
    df.to_csv(output_path, mode = 'a', header = False, index = False)
    
    last_id = data[-1]['post_id']
    
    return last_id, df

---
**Full Script with For Loop Here**<br>
Run this cell to scrape data.

In [285]:
for i in range(1, 2):
    
    base_url = 'https://oauth.reddit.com/r/'
    subreddit = 'RealEstate'

    params = {
        'limit': 100,
        'after': last
    }

    response = requests.get(base_url + subreddit,
                            headers = headers,
                            params = params)
    
    print(f'Scrape: {i}, Status: {response.status_code}')
    
    last, reddit = parse_data(response)
    
    # scrape every 10 seconds
    time.sleep(10)

Scrape: 1, Status: 200


In [286]:
last

't3_1fochf5'

In [289]:
reddit.tail(1)

,post_id,post_title,post_text
99,t3_1fochf5,How to negotiate a mortgage?,Looking to buy \[very\] shortly and trying to ...


In [291]:
# check if duplicate posts
df = pd.read_csv('../data/reddit_realestate.csv')
df.shape

(700, 3)

In [293]:
df['post_id'].nunique()

700